In [1]:
# import dependecies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from scipy import stats
import kagglehub
import json
import os

/Users/mikayla/nutraRx/myenv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
path = kagglehub.dataset_download("shayanhusain/drug-food-interactions-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/mikayla/.cache/kagglehub/datasets/shayanhusain/drug-food-interactions-dataset/versions/1


In [3]:
json_file = os.path.join(path, 'Drug to Food interactions Dataset.json')

In [4]:
with open(json_file, 'r') as f:
    data = json.load(f)
    
data = pd.DataFrame(data)

In [5]:
data.head()

,name,reference,food_interactions
0,Lepirudin,"Knox C, Wilson M, Klinger CM, et al. DrugBank ...",[Avoid herbs and supplements with anticoagulan...
1,Bivalirudin,"Knox C, Wilson M, Klinger CM, et al. DrugBank ...","[Avoid echinacea., Avoid herbs and supplements..."
2,Peginterferon alfa-2a,"Knox C, Wilson M, Klinger CM, et al. DrugBank ...",[Drink plenty of fluids.]
3,Alteplase,"Knox C, Wilson M, Klinger CM, et al. DrugBank ...",[Avoid herbs and supplements with anticoagulan...
4,Interferon alfa-n1,"Knox C, Wilson M, Klinger CM, et al. DrugBank ...",[Avoid alcohol.]


In [6]:
data = data.drop('reference', axis=1)

In [7]:
data['interaction_count'] = data['food_interactions'].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [8]:
data.head()

,name,food_interactions,interaction_count
0,Lepirudin,[Avoid herbs and supplements with anticoagulan...,1
1,Bivalirudin,"[Avoid echinacea., Avoid herbs and supplements...",2
2,Peginterferon alfa-2a,[Drink plenty of fluids.],1
3,Alteplase,[Avoid herbs and supplements with anticoagulan...,1
4,Interferon alfa-n1,[Avoid alcohol.],1


In [9]:
data['food_interactions'] = data['food_interactions'].astype(str)

In [10]:
data['split_sentences'] = data['food_interactions'].apply(lambda x: x.split('.') if isinstance(x, str) else [])

In [11]:
data.head()

,name,food_interactions,interaction_count,split_sentences
0,Lepirudin,['Avoid herbs and supplements with anticoagula...,1,[['Avoid herbs and supplements with anticoagul...
1,Bivalirudin,"['Avoid echinacea.', 'Avoid herbs and suppleme...",2,"[['Avoid echinacea, ', 'Avoid herbs and supple..."
2,Peginterferon alfa-2a,['Drink plenty of fluids.'],1,"[['Drink plenty of fluids, ']]"
3,Alteplase,['Avoid herbs and supplements with anticoagula...,1,[['Avoid herbs and supplements with anticoagul...
4,Interferon alfa-n1,['Avoid alcohol.'],1,"[['Avoid alcohol, ']]"


In [12]:
def findExampleSentences(sentences):
    for sentence in sentences:
        if 'examples include' in sentence.lower() or 'e.g.' in sentence.lower():
            return sentence.strip()
    return ''
data['example_line'] = data['split_sentences'].apply(findExampleSentences)

In [13]:
def extractExamples(text):
    if "Examples include" in text:
        items = text.split('Examples include')[-1]
        return [i.strip().lower() for i in items.split(',')]
    return []
data['examples'] = data['example_line'].apply(extractExamples)

In [14]:
data.head()

,name,food_interactions,interaction_count,split_sentences,example_line,examples
0,Lepirudin,['Avoid herbs and supplements with anticoagula...,1,[['Avoid herbs and supplements with anticoagul...,"Examples include chamomile, garlic, ginger, gi...","[chamomile, garlic, ginger, ginkgo and ginseng]"
1,Bivalirudin,"['Avoid echinacea.', 'Avoid herbs and suppleme...",2,"[['Avoid echinacea, ', 'Avoid herbs and supple...","Examples include garlic, ginger, bilberry, dan...","[garlic, ginger, bilberry, danshen, piracetam,..."
2,Peginterferon alfa-2a,['Drink plenty of fluids.'],1,"[['Drink plenty of fluids, ']]",,[]
3,Alteplase,['Avoid herbs and supplements with anticoagula...,1,[['Avoid herbs and supplements with anticoagul...,"Examples include garlic, ginger, bilberry, dan...","[garlic, ginger, bilberry, danshen, piracetam,..."
4,Interferon alfa-n1,['Avoid alcohol.'],1,"[['Avoid alcohol, ']]",,[]


In [17]:
exploded = data.explode('split_sentences').reset_index(drop=True)

In [15]:
#data.to_csv('drug_to_food_interactions.csv')